In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import re

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import fasttext
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
import warnings
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import neattext.functions as nfx
 
warnings.filterwarnings(action='ignore')

from code_utils.utils import preprocess,aplatir

In [2]:
df_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_ipcc.jsonl', lines= True).dropna(subset=['year'])

In [3]:
wg_doi=pd.read_json('ipcc_wg.json')

In [4]:
df_ipcc=df_ipcc.merge(wg_doi,on='doi',how='left')

In [5]:
df_ipcc.columns

Index(['index', 'doi', 'title', 'year', 'countries', 'concepts', 'sdg',
       'topics', 'is_OA_available', 'title_OA', 'year_OA', 'authors_name',
       'rors', 'institutions_names', 'locations_names', 'locations_ids',
       'test_glutton', 'wg'],
      dtype='object')

In [6]:
dict_wg={'wg1':'science','wg2':'adaptation','wg2_cross':'adaptation','wg3':'mitigation'}

In [7]:
df_ipcc['wg_name']=df_ipcc['wg'].apply(lambda y: [dict_wg[str(x)] for x in y] if isinstance(y, list) else None)

In [8]:
df_not_ipcc=pd.read_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_not_ipcc.jsonl', lines= True)

In [9]:
df_ipcc=df_ipcc[['year','doi', 'title','topics','locations_names','locations_ids','wg_name']]

In [10]:
df_not_ipcc=df_not_ipcc[['year','doi', 'title','topics','locations_names','locations_ids']]

In [11]:
df_ipcc=df_ipcc.dropna(subset=['year','doi', 'title','topics','locations_names','locations_ids']).drop_duplicates(subset=['doi'])

In [12]:
df_not_ipcc=df_not_ipcc.dropna(subset=['doi', 'title','topics','locations_names','locations_ids'])

In [13]:
data_counts=pd.Series([int(x) for x in list(df_ipcc.year.dropna())]).value_counts()
data_counts=pd.Series(dict(data_counts)).sort_index()

building the learning dataset

In [14]:
print(len(df_ipcc),len(df_not_ipcc))

48219 48219


In [15]:
df_all=pd.concat([df_ipcc,df_not_ipcc]).reset_index()
del df_all['index']

In [16]:
df_all.loc[:,'doi']=df_all.loc[:,'doi'].apply(lambda x: str(nfx.remove_stopwords(x)).replace('https://doi.org/','').lower())

In [17]:
df_all.loc[:,'title']=df_all.loc[:,'title'].apply(lambda x: str(nfx.remove_stopwords(x)).replace('}','').replace('{','').lower())

In [18]:
df_all.loc[:,'topics']=df_all.loc[:,'topics'].apply(lambda x: nfx.remove_stopwords(' '.join(x)).lower())

In [19]:
df_all

,year,doi,title,topics,locations_names,locations_ids,wg_name
0,2019.0,10.3389/978-2-88963-118-6,oceanobs'19: ocean opportunity. volume 1,marine biodiversity ecosystem functioning,Frontiers research topics,1664-8714,[science]
1,1900.0,10.1002/andp.19003081208,"\""uber die bedeutung des wasserdampfes und der...",stratospheric chemistry climate change impacts...,Annalen der Physik,0003-3804,[science]
2,1964.0,10.3402/tellusa.v16i1.8885,the parameters atmospheric turbidity,dynamics ocean surface waves wind interaction,Tellus A,0280-6495,[science]
3,1929.0,10.1080/20014422.1929.11880498,on atmospheric transmission sun radiation dust...,aerosols' impact climate hydrological cycle,Geografiska Annaler,1651-3215,[science]
4,2013.0,10.1002/rog.20022,a review global ocean temperature observations...,oceanic modeling circulation studies global se...,Reviews of Geophysics,1944-9208,"[science, adaptation]"
...,...,...,...,...,...,...,...
96433,1967.0,10.1007/bf02349230,played physicians patient's path mental hospital,crisis resolution home treatment teams mental ...,Community mental health journal,0010-3853,NaN
96434,1966.0,10.1038/210701a0,delayed fluorescence chlorophyll vitro vivo,molecular mechanisms photosynthesis photoprote...,Nature,0028-0836,NaN
96435,1966.0,10.1507/endocrine1927.42.8_880,子宮筋homogenateにおけるacetylcholineと組織蛋白との結合に及ぼすoxy...,principles stereochemical structure nomenclatu...,Nippon Naibunpi Gakkai zasshi/Nihon Naibunpi G...,0029-0661,NaN
96436,1686.0,10.1098/rstl.1686.0008,"voyage emperour china eastern tartary, anno 1682",linguistic studies turkic languages silk roads...,Philosophical transactions of the Royal Societ...,0261-0523,NaN


In [20]:
df_all.to_json(module_path+f'\\IPCC_bibliography\\AR6\\structured_data\\data_model_ipcc\\data_model.json')

In [21]:
df_all.columns

Index(['year', 'doi', 'title', 'topics', 'locations_names', 'locations_ids',
       'wg_name'],
      dtype='object')

using fasttext

In [24]:
df_all['label']=df_all['wg_name'].apply(lambda x: ' '.join(['__label__' + str(y) for y in x]) if isinstance(x,list) else '__label__not_ipcc') 
df_all['category_description']=df_all['label']+' '+df_all['title']+' '+df_all['topics']+' '+df_all['locations_names']+' '+df_all['locations_ids']

In [25]:
train, test = train_test_split(df_all, test_size=0.2, random_state=42)

In [26]:
train.shape, test.shape

((77150, 9), (19288, 9))

In [27]:
train.to_csv("teds.train", columns=["category_description"], index=False, header=False)
test.to_csv("teds.test", columns=["category_description"], index=False, header=False)

In [28]:
model = fasttext.train_supervised(input="teds.train")
model.test("teds.test")

(15733, 0.915337189347232, 0.8601720224584877)

In [44]:
model.save_model("fasttext_model_teds.bin")

In [45]:
model = fasttext.load_model("fasttext_model_teds.bin")

In [29]:
test_data = pd.read_csv("teds.test", header=None, names=["text"])

In [30]:
true_labels = []
predicted_labels = []

for line in test_data["text"]:
    label, text = line.split(' ', 1)
    true_labels.append(label)
    text = text.replace('\n', '').strip()
    predicted_label = model.predict(text)[0][0]  
    predicted_labels.append(predicted_label)

In [31]:
conf_matrix = confusion_matrix(true_labels, predicted_labels)

In [32]:
conf_matrix

array([[4429,  383,  165,  260],
       [ 307, 1511,  130,   22],
       [ 127,   68, 9389,   10],
       [ 471,   60,   17, 1939]], dtype=int64)

In [33]:
accuracy_score(true_labels, predicted_labels)

0.8952716715055994

In [34]:
recall_score(true_labels, predicted_labels, average="weighted")

0.8952716715055994

In [35]:
f1_score(true_labels, predicted_labels, average="weighted")

0.894899476808827

In [36]:
t=test.iloc[1,:]

In [65]:
t

year                                                               2017.0
doi                                             10.1007/s00277-017-2971-4
title                   low serum albumin level significant predictor ...
topics                  heparin-induced thrombocytopenia thrombosis pl...
locations_names                                      Annals of hematology
locations_ids                                                   0939-5555
label                                                          __label__0
category_description    __label__0 low serum albumin level significant...
Name: 71796, dtype: object

In [66]:
model.predict(nfx.remove_stopwords(f"{t.title} {t.topics} {t.locations_names} {t.locations_ids}"))

(('__label__0',), array([0.99454683]))

In [67]:
model.predict(nfx.remove_stopwords(f"{t.title} {t.topics} {t.locations_names} {t.locations_ids}"))

(('__label__0',), array([0.99454683]))